# Reinforcement Learning control strategies for Electric Vehicles fleet Virtual Power Plants
Thesis based on the development of a RL agent that manages a VPP through EVs charging stations in an household environment. Main optimization objectives of the VPP are: Valley filling, peak shaving and zero resulting load over time. Main action performed to reach objectives are: storage of Renewable energy resources and power push in the grid at high demand times. The development of the Virtual Power Plant environment is based on the ELVIS (Electric Vehicles Infrastructure Simulator) open library from DAI-Labor: https://github.com/dailab/elvis The thesis code is currently available at: (https://github.com/francescomaldonato/RL_VPP_Thesis)

Author: Francesco Maldonato

## VPP simulator Notebook based on EVs arrival, with StableBaselines3 trained model
(Recurrent PPO) [default: 25 EVs per week simulation]

Installing required packages and dependencies

In [1]:
%%capture
!pip install py-elvis==0.2.1
!pip install pyyaml==5.4
!pip install plotly==5.9.0
!pip install -U kaleido==0.2.1

!pip install stable-baselines3[extra]==1.6.1
!pip install stable-baselines==1.6.1
!pip install sb3-contrib==1.6.1
!pip install gym==0.20.0
!pip install -q wandb==0.13.4

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/DAI-Labor/RL_VPP_Thesis/
%ls

Mounted at /content/drive
/content/drive/MyDrive/DAI-Labor/RL_VPP_Thesis
Agent_trainer_notebooks/          __pycache__/
Algorithm_simulator_notebooks/    README.md
data/                             trained_models/
EV_experiment_notebooks/          VPP_environment.py
Hyperparameters_sweep_notebooks/  VPP_simulator.ipynb
log.log                           wandb/


In [3]:
# #Cloning repository and changing directory
# !git clone https://github.com/francescomaldonato/RL_VPP_Thesis.git
# %cd RL_VPP_Thesis/
# %ls

In [4]:
import yaml
import numpy as np
from gym import Env
from VPP_environment import VPPEnv, VPP_Scenario_config
from elvis.config import ScenarioConfig
import os
import torch
import random
import wandb
from sb3_contrib import RecurrentPPO #The available algoritmhs in sb3-contrib for the custom environment with MultiInputPolicy
from sb3_contrib.common.maskable.utils import get_action_masks
import stable_baselines3 as sb3
from stable_baselines3.common.env_checker import check_env

#Check if cuda device is available for training
print("Torch-Cuda available device:", torch.cuda.is_available())
print(sb3.get_system_info())
#!wandb --version

Torch-Cuda available device: False
OS: Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic #1 SMP Fri Aug 26 08:44:51 UTC 2022
Python: 3.7.15
Stable-Baselines3: 1.6.1
PyTorch: 1.12.1+cu113
GPU Enabled: False
Numpy: 1.21.6
Gym: 0.20.0

({'OS': 'Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic #1 SMP Fri Aug 26 08:44:51 UTC 2022', 'Python': '3.7.15', 'Stable-Baselines3': '1.6.1', 'PyTorch': '1.12.1+cu113', 'GPU Enabled': 'False', 'Numpy': '1.21.6', 'Gym': '0.20.0'}, 'OS: Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic #1 SMP Fri Aug 26 08:44:51 UTC 2022\nPython: 3.7.15\nStable-Baselines3: 1.6.1\nPyTorch: 1.12.1+cu113\nGPU Enabled: False\nNumpy: 1.21.6\nGym: 0.20.0\n')


In [5]:
# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

## Load ELVIS YAML config file
Section where the EVs arrival simulation parameters are loaded through the Yaml config file from the 'data/config_builder/' folder.

In [6]:
#Loading paths for input data
current_folder = ''
VPP_training_data_input_path = current_folder + 'data/data_training/environment_table/' + 'Environment_data_2019.csv'
VPP_testing_data_input_path = current_folder + 'data/data_testing/environment_table/' + 'Environment_data_2020.csv'
VPP_validating_data_input_path = current_folder + 'data/data_validating/environment_table/' + 'Environment_data_2018.csv'
elvis_input_folder = current_folder + 'data/config_builder/'

#case = 'wohnblock_household_simulation_adaptive.yaml' #(loaded by default, 20 EVs arrivals per week with 50% average battery)

#Try different simulation parameters, uncomment below
#case = 'wohnblock_household_simulation_adaptive_10.yaml' #(10 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_15.yaml' #(15 EVs arrivals per week with 50% average battery)
case = 'wohnblock_household_simulation_adaptive_25.yaml' #(25 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_30.yaml' #(30 EVs arrivals per week with 50% average battery) 
#case = 'wohnblock_household_simulation_adaptive_35.yaml' #(35 EVs arrivals per week with 50% average battery) 

with open(elvis_input_folder + case, 'r') as file:
    yaml_str = yaml.full_load(file)

elvis_config_file = ScenarioConfig.from_yaml(yaml_str)
VPP_config_file = VPP_Scenario_config(yaml_str)

print(elvis_config_file)
print(VPP_config_file)

Vehicle types: <generator object ScenarioConfig.__str__.<locals>.<genexpr> at 0x7faa26a874d0>Mean parking time: 23.99
Std deviation of parking time: 1
Mean value of the SOC distribution: 0.5
Std deviation of the SOC distribution: 0.1
Max parking time: 24
Number of charging events per week: 25
Vehicles are disconnected only depending on their parking time
Queue length: 0
Opening hours: None
Scheduling policy: Uncontrolled

{'start_date': '2022-01-01T00:00:00', 'end_date': '2023-01-01T00:00:00', 'resolution': '0:15:00', 'num_households': 4, 'solar_power': 16, 'wind_power': 12, 'EV_types': [{'battery': {'capacity': 100, 'efficiency': 1, 'max_charge_power': 150, 'min_charge_power': 0}, 'brand': 'Tesla', 'model': 'Model S', 'probability': 1}], 'charging_stations_n': 4, 'EVs_n': 25, 'EVs_n_max': 1305, 'mean_park': 23.99, 'std_deviation_park': 1, 'EVs_mean_soc': 50.0, 'EVs_std_deviation_soc': 10.0, 'EV_load_max': 44, 'EV_load_rated': 14.8, 'EV_load_min': 1, 'houseRWload_max': 10, 'av_max_ener

In [7]:
#TESTING Environment initialization
env = VPPEnv(VPP_testing_data_input_path, elvis_config_file, VPP_config_file)
env.plot_Dataset_autarky()

Charging event: 1, Arrival time: 2022-01-01 00:15:00, Parking_time: 24, Leaving_time: 2022-01-02 00:15:00, SOC: 0.4108807863621144, SOC target: 1.0, Connected car: Tesla, Model S 
 ... 
 Charging event: 1304, Arrival time: 2022-12-31 15:30:00, Parking_time: 23.11044655946803, Leaving_time: 2023-01-01 14:36:37.607614, SOC: 0.556624340258615, SOC target: 1.0, Connected car: Tesla, Model S 

-DATASET: House&RW_energy_sum=kWh  -21214.64 , Grid_used_en(grid-import)=4947.18kWh , autarky-rate=83.0 , RE-to-vehicle_unused_en(grid-export)=-26161.81kWh , self-consump.rate=47.9 , Total_selling_cost=€  -489.75 , Grid_cost=€  233.11
- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  27138.02 , Grid_used_en=kWh  43446.92 , RE-to-vehicle_unused_en=kWh  16308.9 , Total_selling_cost=€  1072.33 , Grid_cost=€  1539.54 , Charging_events=  1304 
- Exp.VPP_goals: Grid_used_en=kWh 0, RE-to-vehicle_unused_en=kWh 0, Grid_cost=€ 0 , Av.EV_en_left=kWh  70.06


In [8]:
env.plot_VPP_input_data()

Output hidden; open in https://colab.research.google.com to view.

In [9]:
env.plot_ELVIS_data()

In [10]:
#Function to check custom environment and output additional warnings if needed
check_env(env)
env.plot_reward_functions()

- ELVIS.Simulation (Av.EV_SOC=  50.0 %):
 Sum_Energy=kWh  27734.61 , Grid_used_en(grid-import)=44478.41kWh , Total_demand=77968.7kWh , autarky-rate=0.4 , RE-to-vehicle_unused_en(grid-export)=16743.8kWh , Total_supply=50234.1kWh , self-consump.rate=0.7 , Grid_cost=€  1568.26 , Total_selling_cost=€  1075.43 , Av.EV_en_left=kWh  100.0 , Charging_events=  1304 
- VPP_goal_upper_limit: Grid_used_en=kWh 0, RE-to-vehicle_unused_en=kWh 0, Grid_cost=€ 0 , Av.EV_en_left=kWh  70.06
Simulating VPP....


### VPP Simulation test with random actions [no model loaded]

In [11]:
episodes = 1
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    while not done:
        action_masks = get_action_masks(env)
        action = env.action_space.sample()
        
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

VPP_table = env.VPP_table
env.plot_VPP_energies()

Output hidden; open in https://colab.research.google.com to view.

In [12]:
#env.plot_Elvis_results()

In [13]:
#env.plot_VPP_results()

In [14]:
env.plot_VPP_autarky()

In [15]:
env.plot_VPP_supply_demand()

Output hidden; open in https://colab.research.google.com to view.

In [16]:
env.plot_VPP_Elvis_comparison()

In [17]:
env.plot_rewards_results()

Output hidden; open in https://colab.research.google.com to view.

In [18]:
#env.plot_rewards_stats()

In [19]:
env.plot_EVs_kpi()

In [20]:
env.plot_actions_kpi()

In [21]:
env.plot_load_kpi()

In [22]:
env.plot_yearly_load_log()

TypeError: ignored

In [ ]:
## Wandb login to load models
#In Colab, uncomment below:
# %env "WANDB_DISABLE_CODE" True
# %env "WANDB_NOTEBOOK_NAME" "VPP_simulator.ipynb"
# os.environ['WANDB_NOTEBOOK_NAME'] = 'VPP_simulator.ipynb'
#wandb.login(relogin=True)

#In local notebook, uncomment below:
#your_wandb_login_code = 0123456789abcdefghijklmnopqrstwxyzàèìòù0 #example length
#!wandb login {your_wandb_login_code}

In [ ]:
#Loading training model, from local directory or from wandb previous trainings
RecurrentPPO_path = "trained_models/RecurrentPPO_models/model_RecurrentPPO_"

#model_id = "s37o8q0n"
model_id = "333ckz0i"
model = RecurrentPPO.load(RecurrentPPO_path + model_id, env=env)

# run_id_restore = "2y2dqvyn"
# model = wandb.restore(f'model_{run_id_restore}.zip', run_path=f"francesco_maldonato/RL_VPP_Thesis/{run_id_restore}")

## Testing dataset VPP Simulation using the loaded trained model

In [ ]:
#TEST Model
episodes = 1
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    while not done:
        #env.render()
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True) #Now using our trained model with deterministic prediction [should improve performances]
        env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

#Save the VPP table
VPP_table = env.save_VPP_table(save_path='data/environment_optimized_output/VPP_table.csv')

In [ ]:
env.plot_VPP_energies()

In [ ]:
VPP_table.head(15000)

In [ ]:
#env.plot_Elvis_results()

In [ ]:
#env.plot_VPP_results()

In [ ]:
env.plot_VPP_autarky()

In [ ]:
env.plot_VPP_supply_demand()

In [ ]:
env.plot_VPP_Elvis_comparison()

In [ ]:
env.plot_rewards_results()

In [ ]:
env.plot_rewards_stats()

In [ ]:
env.plot_EVs_kpi()

In [ ]:
env.plot_load_kpi()

In [ ]:
env.plot_yearly_load_log()

## Validating dataset VPP Simulation using the loaded trained model

In [ ]:
#VALIDATING Environment initialization
env = VPPEnv(VPP_validating_data_input_path, elvis_config_file, VPP_config_file)

In [ ]:
#Function to check custom environment and output additional warnings if needed
check_env(env)
plot_VPP_input_data = env.plot_VPP_input_data()
plot_VPP_input_data.show()

In [ ]:
#model = PPO.load(PPO_path + model_run_ID, env = env)
model = RecurrentPPO.load(RecurrentPPO_path + model_id, env=env)

In [ ]:
#TEST Model
episodes = 1
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    while not done:
        #env.render()
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True) #Now using our trained model with deterministic prediction [should improve performances]
        env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

VPP_table = env.VPP_table
#print(env.lstm_states_list)

In [ ]:
env.plot_VPP_energies()

In [ ]:
VPP_table.head(15000)

In [ ]:
#env.plot_VPP_results()


In [ ]:
env.plot_VPP_autarky()

In [ ]:
env.plot_VPP_supply_demand()

In [ ]:
env.plot_VPP_Elvis_comparison()

In [ ]:
env.plot_rewards_results()

In [ ]:
env.plot_rewards_stats()

In [ ]:
env.plot_EVs_kpi()

In [ ]:
env.plot_load_kpi()

In [ ]:
env.plot_yearly_load_log()

## Training dataset VPP Simulation using the loaded trained model

In [ ]:
#TRAINING Environment initialization
env = VPPEnv(VPP_training_data_input_path, elvis_config_file, VPP_config_file)

In [ ]:
#Function to check custom environment and output additional warnings if needed
check_env(env)
plot_VPP_input_data = env.plot_VPP_input_data()
plot_VPP_input_data.show()

In [ ]:
#model = PPO.load(PPO_path + model_run_ID, env = env)
model = RecurrentPPO.load(RecurrentPPO_path + model_id, env=env)

In [ ]:
#TEST Model
episodes = 1
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    # cell and hidden state of the LSTM
    lstm_states = None
    num_envs = 1
    # Episode start signals are used to reset the lstm states
    episode_starts = np.ones((num_envs,), dtype=bool)
    while not done:
        #env.render()
        action_masks = get_action_masks(env)
        action, lstm_states = model.predict(obs, state=lstm_states, episode_start=episode_starts, deterministic=True) #Now using our trained model with deterministic prediction [should improve performances]
        env.lstm_state = lstm_states
        obs, reward, done, info = env.step(action)
        episode_starts = done
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

VPP_table = env.VPP_table
#print(env.lstm_states_list)

In [ ]:
env.plot_VPP_energies()

In [ ]:
VPP_table.head(14995)

In [ ]:
#env.plot_VPP_results()

In [ ]:
env.plot_VPP_autarky()

In [ ]:
env.plot_VPP_supply_demand()

In [ ]:
env.plot_VPP_Elvis_comparison()

In [ ]:
env.plot_rewards_results()

In [ ]:
env.plot_rewards_stats()

In [ ]:
env.plot_EVs_kpi()

In [ ]:
env.plot_actions_kpi()

In [ ]:
env.plot_load_kpi()

In [ ]:
env.plot_yearly_load_log()

In [ ]:
#env.close()